In [1]:
def load_csv(filename):
    with open(filename, 'r', encoding='utf-8-sig') as f:
        csv_list = [row.strip().split(',') for row in f.readlines()]
    return csv_list

def parse_row_as_element(row):
    name = row[0].lower()
    prevElemCombos = row[1].lower().split(' / ')
    prevElemCombos = [p.lower().split('+') for p in prevElemCombos]
    if prevElemCombos == [['available from start.']]:
        prevElemCombos = [['BASE_ELEMENT']]
    # return Element(name, prevElemCombos)
    return {name : {'name': name, 'prevElemCombos': prevElemCombos, 'nextPossibleElements': []}}

def printx(listx):
    for x in listx:
        print(x)

def create_elements(csv_list):
    elements_arr = [parse_row_as_element(row) for row in csv_list]
    elements = elements_arr[0]
    for e_arr in elements_arr[1:]:
        elements.update(e_arr)
    return elements

In [2]:
filename = 'combinations_list_orig.csv'
csv_list = load_csv(filename)
elements = create_elements(csv_list)
# print(elements)

In [3]:
def create_tree(elements):
    elem_names = list(elements.keys())
    elements['BASE_ELEMENT'] = {'name': 'BASE_ELEMENTS', 'prevElemCombos': [], 'nextPossibleElements': []}
    for elem in elem_names:
        for prevElemCombo in elements[elem]['prevElemCombos']:
            for prevElem in prevElemCombo:
                try:
                    elements[prevElem]['nextPossibleElements'].append(elem)
                except KeyError:
                    elements[prevElem] = {'name': prevElem, 'prevElemCombos': [], 'nextPossibleElements': [elem]}         

    return elements 

elements = create_tree(elements)

In [4]:
print(elements['BASE_ELEMENT'])

{'name': 'BASE_ELEMENTS', 'prevElemCombos': [], 'nextPossibleElements': ['air', 'earth', 'fire', 'water']}


In [5]:
from functools import lru_cache
cache = lru_cache(maxsize=None)

def get_possible_count(element, elements_subset, elements, count_level=1):

    count = 0
    elements_subset.append(element)
    count += len(get_possible_elements(elements_subset, elements, count_level))
    elements_subset.pop()
    return count

def get_possible_elements(elements_subset, elements, count_level=1):
    import itertools
    results = []
    for elem in elements:
        for prevElemCombo in elements[elem]['prevElemCombos']:
            if len(set(prevElemCombo) & (set(elements_subset))) == len(prevElemCombo):
                results.append([prevElemCombo, elem])
    
    if count_level <0:
        ''' shuffle and return result'''
        import random
        random.shuffle(results)
        return results

    if count_level == 0:
        for r in results:
            r.append(1)
    else:
        for r in results:
            r.append(get_possible_count(r[1], elements_subset, elements, count_level-1))

    ''' sort results by count '''
    results.sort(key=lambda x: x[-1], reverse=True)
    return results
        

    

In [6]:
starting_elements = elements['BASE_ELEMENT']['nextPossibleElements']
 
possible_elements = get_possible_elements(starting_elements, elements, count_level=1)

In [7]:
printx(possible_elements)

[['air', 'fire'], 'energy', 9]
[['earth', 'fire'], 'lava', 9]
[['water', 'fire'], 'steam', 8]
[['earth', 'air'], 'dust', 7]
[['water', 'earth'], 'mud', 7]
[['water', 'air'], 'rain', 7]


In [8]:
new_elements = starting_elements

In [9]:
moves_list = []
from tqdm import tqdm

for I in tqdm(range(70)):
    current_elements = new_elements
    possible_elements_arr = get_possible_elements(current_elements, elements, 2 if I < 10 else -1)
    moves_list.append(possible_elements_arr[0])
    possible_elements = [elemx[1] for elemx in possible_elements_arr]
    new_elements = current_elements+[possible_elements[0]]
printx(moves_list)
printx(new_elements)

100%|██████████| 70/70 [00:05<00:00, 12.23it/s]

[['air', 'fire'], 'energy', 9]
[['earth', 'fire'], 'lava', 12]
[['water', 'fire'], 'steam', 14]
[['air', 'steam'], 'cloud', 17]
[['air', 'cloud'], 'sky', 19]
[['air', 'lava'], 'stone', 22]
[['fire', 'stone'], 'metal', 26]
[['air', 'stone'], 'sand', 29]
[['fire', 'sand'], 'glass', 35]
[['metal', 'energy'], 'electricity', 39]
[['sand', 'electricity'], 'glass']
[['sand', 'energy'], 'sandstorm']
[['stone', 'sand'], 'sandstone']
[['air', 'fire'], 'energy']
[['energy', 'cloud'], 'storm']
[['earth', 'energy'], 'earthquake']
[['cloud', 'electricity'], 'storm']
[['earth', 'energy'], 'earthquake']
[['electricity', 'metal'], 'wire']
[['air', 'lava'], 'stone']
[['fire', 'stone'], 'metal']
[['glass', 'metal'], 'glasses']
[['air', 'steam'], 'cloud']
[['sky', 'earth'], 'horizon']
[['air', 'energy'], 'wind']
[['glass', 'metal'], 'glasses']
[['air', 'metal'], 'rust']
[['earth', 'energy'], 'earthquake']
[['water', 'fire'], 'steam']
[['earthquake', 'earth'], 'mountain']
[['metal', 'stone'], 'blade']
[['l